## Introduction

- 2 states representations
- Motivations of using function approximation

- Our states, actions, and rewards

## Agents

- Random vs. Random

- For each agent: 
    - agent vs. random
    - winning rate
    - learning curve

Play against each other
- Table

## Discussion & Limitation

1. Hyperparameters & Architecture
2. Multiple players
3. Two more base agents
4. State representation
5. Game is difficult
